In [1]:
import config.mongo_setup as mongo_setup
from models import Tweets, Replies, RepliesSentiment, Summarizations
import pandas as pd
import yaml
import requests
import json
mongo_setup.global_init()

In [17]:

def connect_azure():
    '''Gets Azure auth data'''
    auth = process_yaml()
    azure_url = auth["azure"]["endpoint"]
    sentiment_url = f"{azure_url}text/analytics/v3.1/sentiment"
    summarization_url = "https://tcc-sentiment-analysis.cognitiveservices.azure.com/text/analytics/v3.2-preview.1/analyze/jobs/185e3958-d8b8-4440-b4d2-8357742fa0b9"
    subscription_key = auth["azure"]["subscription_key"]
    azure_header = {"Ocp-Apim-Subscription-Key": subscription_key}
    return sentiment_url, azure_header, summarization_url


def process_yaml():
    with open("config/config.yaml") as file:
        return yaml.safe_load(file)

def save_file(json_dict,name):
    with open(f"{name}.json", "w") as out_file:
        json.dump(json_dict, out_file, indent=4)

sentiment_url, azure_header,summarization_url = connect_azure()

In [18]:
response = requests.request("GET", summarization_url, headers=azure_header)
print(response)
print(response.json())
save_file(response.json(),'summarization_test')

<Response [200]>
{'jobId': '185e3958-d8b8-4440-b4d2-8357742fa0b9', 'lastUpdateDateTime': '2021-09-20T18:03:54Z', 'createdDateTime': '2021-09-20T18:03:37Z', 'expirationDateTime': '2021-09-21T18:03:37Z', 'status': 'succeeded', 'errors': [], 'displayName': 'NA', 'tasks': {'completed': 1, 'failed': 0, 'inProgress': 0, 'total': 1, 'extractiveSummarizationTasks': [{'lastUpdateDateTime': '2021-09-20T18:03:54.0166739Z', 'taskName': 'ExtractiveSummarization_latest', 'state': 'succeeded', 'results': {'documents': [{'id': '1', 'sentences': [{'text': 'bolsonaro, você é apaixonado pelo lula???', 'rankScore': 0.97, 'offset': 14, 'length': 41}, {'text': 'o mbl juntamente com joão dória e mais meia dúzia de inconformados fizeram manifestação contra bolsonaro.', 'rankScore': 0.9, 'offset': 57, 'length': 105}, {'text': 'com bastante distanciamento social.', 'rankScore': 0.86, 'offset': 163, 'length': 35}, {'text': 'bem distante mesmo...', 'rankScore': 0.81, 'offset': 199, 'length': 21}, {'text': 'foraaa

In [19]:
objs_replies = Replies.objects.filter(conversation_id=1437054980884684804).values_list('id','reply_id', 'in_reply_to_user_id', 'conversation_id' ,'author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count')
replies_df = pd.DataFrame.from_records(data=objs_replies, columns=['id','reply_id', 'in_reply_to_user_id', 'conversation_id' ,'author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count'])
len(replies_df)



250

In [10]:
summ = Summarizations.objects.all().values_list('summarization_id','text', 'rankScore')
df = pd.DataFrame(data=summ, columns=['summarization_id','text', 'rankScore'])
df

,summarization_id,text,rankScore
0,1437054980884684804,""" vagabundo o próprio lobo, achando que t...",1.00
1,1437054980884684804,"melhora esse marketing, pq esse de bom moço n...",0.89
2,1437054980884684804,"luladrao pensa que engana a quem , esse infe...",0.77
3,1437054980884684804,ao menos dessa vez segurou o livro corretamente.,0.65
4,1437054980884684804,já aprendeu a ler???,0.62
5,1437054980884684804,🤣🤣🤣🤣🤣🤣 nosso verdadeiro presidente indican...,0.53
6,1437054980884684804,em entrevista já disse que odeia ler...,0.42
7,1437054980884684804,falou em deus é por que tá próximo das eleiçõe...,0.35
8,1437054980884684804,kkkk tô falando que este demônio findará num...,0.33
9,1437054980884684804,kkkkkkkkkkkkkkkkkkk kkkkkkkkkkkkkkkkkkk kkkkkk...,0.42


In [83]:

df = pd.DataFrame(data=summ, columns=['summarization_id','sentences',])
df = df.explode('sentences')
df = pd.concat([df, df['sentences'].apply(pd.Series)], axis=1)
df = df.drop(columns='sentences')
df = df.rename(columns={0:'Text',1:'rankScore'})
# df = pd.json_normalize(df['sentences'])
df

,summarization_id,Text,rankScore
0,1437054980884684804,text,rankScore
0,1437054980884684804,text,rankScore
0,1437054980884684804,text,rankScore
0,1437054980884684804,text,rankScore
0,1437054980884684804,text,rankScore
0,1437054980884684804,text,rankScore
0,1437054980884684804,text,rankScore
0,1437054980884684804,text,rankScore
0,1437054980884684804,text,rankScore
0,1437054980884684804,text,rankScore


In [21]:

objs = RepliesSentiment.objects.all()
text = ''
for obj in objs:
    text = text + ' ' + obj.clean_text

print(len(text))

1600121


In [12]:

tweets = Tweets.objects(tweet_id=1437054980884684804).first()
tweets.text

'Recomendação de leitura para o domingo: Amor à Maneira de Deus, do meu amigo Padre Júlio Lancellotti (@pejulio). Lições sobre amor, compaixão e humanismo. \n\nFoto: Ricardo Stuckert https://t.co/2PQ1Yza9ab'

In [ ]:
# response.json()['tasks']['extractiveSummarizationTasks'][0]['results']['documents'][0]['id']
# response.json()['tasks']['extractiveSummarizationTasks'][0]['results']['documents'][0]['setences']


# response.json()['tasks']['extractiveSummarizationTasks'][0]['sentences'][i]['text']
# response.json()['tasks']['extractiveSummarizationTasks'][0]['sentences'][i]['rankScore']